In [1]:
import pandas as pd
import numpy as np
import os, sys
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, MaxPooling2D, AveragePooling2D, BatchNormalization, Conv2D
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.metrics import categorical_crossentropy
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
df = pd.read_csv('Emotions.csv')
display(df[0:5])

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [4]:
x_train, x_test, y_train, y_test = [], [], [], []

In [5]:
for index, row in df.iterrows():
    val = row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
            x_train.append(np.array(val, 'float32'))
            y_train.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
            x_test.append(np.array(val, 'float32'))
            y_test.append(row['emotion'])
    except :
        print(f"error occured in row: {row} index: {index}")
    

In [8]:
x_train = np.array(x_train, 'float32')
x_test = np.array(x_test, 'float32')
y_train = np.array(y_train, 'float32')
y_test = np.array(y_test, 'float32')

In [10]:
x_train.shape

(28709, 2304)

In [11]:
x_train = x_train[0:5000]
x_test = x_test[0:1000]
y_train = y_train[0:5000]
y_test = y_test[0:1000]

In [12]:
x_train.shape

(5000, 2304)

In [13]:
x_test.shape

(1000, 2304)

In [15]:
y_train[5]

2.0

In [17]:
y_train = np_utils.to_categorical(y_train, num_classes=7)
y_test = np_utils.to_categorical(y_test, num_classes=7)

In [20]:
y_test[0]

array([1., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [18]:
x_train -= np.mean(x_train, axis=0)
x_train /= np.std(x_train, axis=0)

In [19]:
x_test -= np.mean(x_test, axis=0)
x_test /= np.std(x_test, axis=0)

In [21]:
x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)

In [22]:
x_train.shape

(5000, 48, 48, 1)

In [24]:
from keras.layers import Dropout

In [25]:
model = Sequential()
model.add(Conv2D(64, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))


In [26]:
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.5))

In [27]:
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Flatten())

In [28]:
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

In [29]:
model.add(Dense(7, activation='softmax'))

In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 44, 44, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 22, 22, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 22, 22, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 20, 20, 64)        36928     
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 18, 18, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 9, 9, 64)         

In [31]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [33]:
model.fit(x_train, y_train, batch_size=64, epochs=50, shuffle=True, validation_data= (x_test, y_test), verbose=2)

Train on 5000 samples, validate on 1000 samples
Epoch 1/50
 - 53s - loss: 1.7601 - accuracy: 0.2702 - val_loss: 1.7650 - val_accuracy: 0.2680
Epoch 2/50
 - 52s - loss: 1.7389 - accuracy: 0.2928 - val_loss: 1.7360 - val_accuracy: 0.2960
Epoch 3/50
 - 52s - loss: 1.7114 - accuracy: 0.3026 - val_loss: 1.7158 - val_accuracy: 0.3140
Epoch 4/50
 - 52s - loss: 1.6782 - accuracy: 0.3364 - val_loss: 1.6823 - val_accuracy: 0.3130
Epoch 5/50
 - 53s - loss: 1.6535 - accuracy: 0.3370 - val_loss: 1.6694 - val_accuracy: 0.3350
Epoch 6/50
 - 53s - loss: 1.6236 - accuracy: 0.3646 - val_loss: 1.6276 - val_accuracy: 0.3440
Epoch 7/50
 - 53s - loss: 1.5937 - accuracy: 0.3754 - val_loss: 1.6283 - val_accuracy: 0.3520
Epoch 8/50
 - 53s - loss: 1.5793 - accuracy: 0.3862 - val_loss: 1.6021 - val_accuracy: 0.3680
Epoch 9/50
 - 53s - loss: 1.5638 - accuracy: 0.3944 - val_loss: 1.5923 - val_accuracy: 0.3550
Epoch 10/50
 - 53s - loss: 1.5386 - accuracy: 0.4022 - val_loss: 1.5939 - val_accuracy: 0.3610
Epoch 11/50

In [34]:
fer_json = model.to_json()
with open("emotion.json",'w') as json_file:
    json_file.write(fer_json)
model.save_weights('emotion_weights.h5')